# 不含模型参数的自定义层

自定义一个将输入减掉均值后输出的层，并将层的计算定义在forward函数里

In [1]:
import torch
from torch import nn

class CenterLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x-x.mean()

实例化此层，然后做前向计算

In [2]:
layer=CenterLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更加复杂的模型

In [3]:
net=nn.Sequential(nn.Linear(8,128),CenterLayer())

因为减去均值，所以均值接近于0

In [4]:
y = net(torch.rand(4, 8))
y.mean().item()

5.587935447692871e-09

# 含模型参数的自定义层